In [1]:
from utils import load_chroma_pdf, word_wrap
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import const
import json
import requests

In [2]:
embedding_function = SentenceTransformerEmbeddingFunction()

chroma_collection = load_chroma_pdf(collection_name='boe_normativa_trafico', 
                                    persist_path='/teamspace/studios/this_studio/ProjectGenAI/data/chroma_databases')
chroma_collection.count()

11566

In [3]:
original_query = "Si no me paro a socorrer en caso de accidente, que me puede pasar?"

## Basic retrieval

In [4]:
results = chroma_collection.query(query_texts=[original_query], n_results=5)
retrieved_documents = results['documents'][0]

## Query expansion

### Expansion generated

In [19]:
hypothetical_answer = augment_generated_query(original_query)
print(hypothetical_answer)
joint_query = f"{original_query} {hypothetical_answer}"

results = chroma_collection.query(query_texts=joint_query, n_results=5, include=['documents', 'embeddings'])

# Final documents passed to the chatbot function
retrieved_documents = results['documents'][0]

Según la normativa de tráfico de la Dirección General de Tráfico (DGT), como peatón no puedes circular por las autopistas, autovías, vías interurbanas y túneles, salvo que exista una señalización específica que lo permita. Es importante recordar que los peatones deben circular por las aceras, pasos de peatones y zonas habilitadas para ellos, respetando siempre las normas de tráfico y priorizando su seguridad.


### Expansion multiple queries

In [28]:
augmented_queries = augment_multiple_query(original_query)

'''
for query in augmented_queries:
    print(query)
'''

queries = [original_query] + augmented_queries
results = chroma_collection.query(query_texts=queries, n_results=5, include=['documents', 'embeddings'])
retrieved_documents = results['documents']

# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

# Final documents passed to the chatbot function
retrieved_documents = unique_documents

'''
for i, documents in enumerate(retrieved_documents):
    print(f"Query: {queries[i]}")
    print('')
    print("Results:")
    for doc in documents:
        print(word_wrap(doc))
        print('')
    print('-'*100)
'''

¿Cuál es el procedimiento a seguir al tener un accidente de tráfico? 
¿Qué información debo recopilar en caso de presenciar un accidente de tráfico? 
¿Cuál es la forma correcta de ayudar a las víctimas de un accidente de tráfico? 
¿Qué debo hacer si soy testigo de un accidente de tráfico pero no estoy involucrado en él? 
¿Cuál es la manera adecuada de alertar a los servicios de emergencia en caso de presenciar un accidente de tráfico?


'\nfor i, documents in enumerate(retrieved_documents):\n    print(f"Query: {queries[i]}")\n    print(\'\')\n    print("Results:")\n    for doc in documents:\n        print(word_wrap(doc))\n        print(\'\')\n    print(\'-\'*100)\n'

In [5]:
answer = chatbot(original_query,retrieved_documents)
print(answer)

NameError: name 'chatbot' is not defined